# Projeto 3 - CD 
Eli Jose, Pedro Greco, Eduardo Vaz

### Como prever e evitar de maneira mais eficiente que os funcionários de uma empresa se demitam prematuramente?

Em muitas empresas o número de funcionários saindo prematuramente e sem motivo claro é considerável. Quando um empregado que tinha um bom histórico de trabalho e eficiencia deixa a empresa, ele atrapalha no desenvolvimento e continuidade dos processos que ele exercia. Desse modo, é de grande importância a criação de uma ferramenta que permita a realização de uma análise e predição de quais funcionários deixariam seu trabalho. Por isso, objetivo desse projeto é criar um algoritmo que exerça essa função da maneira mais precisa o possível. 

Para isso, foram utilizados conhecimentos sobre Python e sobre Regressão Linear e Logística para programar um código que consegue prever quais funcionários sairão da empresa considerando suas informações e históricos. Assim, foi utilizada uma base de dados com vários aspectos sobre cada empregado, suas características e se ele saiu da empresa ou não, para 'ensinar' o programa a conseguir prever futuras demissões próprias de funcionários.

Em aula aprendemos a lógica e explicação de Regressão Linear. Desse modo, não tinhamos conhecimentos suficientes sobre a Regressão Logística necessários para tratar esse problema. Por isso, utilizamos um tutorial sobre como montar uma regressão logística disponível em:
 http://nbviewer.jupyter.org/gist/justmarkham/6d5c061ca5aee67c4316471f8c2ae976

##### Regressão Linear e Regressão Logística

A regressão linear é uma equação criada a partir da relação entre duas variáveis continuas para estimar o valor esperado de uma variável y, considerando o valor de outras variáveis x. Desse modo, a regressão logistica, usada nesse projeto, seria uma equação que estimaria o valor de uma variável y, sendo está dictômica, ou seja, tem apenas dois possíveis valores, diferentemente da regressão linear em que y é continua e tem infinitos possíveis valores.

Então, como neste caso o objetivo é prever se o funcionario sairá ou não da empresa (duas possibilidades) a partir de uma série de variáveis pessoais do empregado, a regressão logística foi utilizada. As variáveis x utilizadas foram as seguintes:

- Nível de Satisfação do Funcionário
- Avaliação do Funcionário
- Número de projetos do Funcionário
- Média de horas gastas no trabalho
- Tempo gasto na empresa
- Teve ou não um acidente de trabalho
- Teve ou não uma promoção nos últimos 5 anos
- Departamento de trabalho do Funcionário
- Salário do Funcionário

Todas essas variáveis estão presentes em um dataframe retirado de https://www.kaggle.com/ludobenistant/hr-analytics/data . Este foi utilizado como treinamento para que fossem obtidos os valores de coeficientes que permitiriam prever se o funcionário sairia da empresa ou não. Assim, ao final o algoritmo será capaz de informar se um empregado sairá ou não da empresa baseado nessas características individuais dele.

In [331]:
#importação das bibliotecas necessárias
import math
import os.path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.svm import LinearSVC
from sklearn import metrics

In [332]:
#importação do dataframe e renomeação das colunas

data = pd.read_csv('dd1.csv',sep=',')
data = data.rename(columns = {'satisfaction_level':'satisfacao'})
data = data.rename(columns = {'last_evaluation':'evaluacao'}) 
data = data.rename(columns = {'number_project':'projetos'}) 
data = data.rename(columns = {'average_montly_hours':'hrsmedia'}) 
data = data.rename(columns = {'time_spend_company':'tempogasto'})
data = data.rename(columns = {'left':'saiu'})
data = data.rename(columns = {'promotion_last_5years':'promocoes'})
data = data.rename(columns = {'sales':'setor'})
data = data.rename(columns = {'salary':'salario'})
data = data.rename(columns = {'Work_accident':'acidentes'})


#trocando o data type da categoria salario para se adaptar a biblioteca sklearn
data['salario'].astype(str)

data.loc[(data.salario== 'low'), 'salario'] = 1
data.loc[(data.salario== 'medium'), 'salario'] = 2
data.loc[(data.salario== 'high'), 'salario'] = 3

#trocando o data type da categoria setor para se adaptar a biblioteca sklearn
data['setor'].astype(str)
data.loc[(data.setor== 'sales'), 'setor'] = 0
data.loc[(data.setor== 'accounting'), 'setor'] = 1
data.loc[(data.setor== 'hr'), 'setor'] = 2
data.loc[(data.setor== 'technical'), 'setor'] = 3
data.loc[(data.setor== 'support'), 'setor'] = 4
data.loc[(data.setor== 'management'), 'setor'] = 5
data.loc[(data.setor== 'IT'), 'setor'] = 6
data.loc[(data.setor== 'product_mng'), 'setor'] = 7
data.loc[(data.setor== 'marketing'), 'setor'] = 8

data.head()





,satisfacao,evaluacao,projetos,hrsmedia,tempogasto,acidentes,saiu,promocoes,setor,salario
0,0.38,0.53,2,157,3,0,1,0,0,1
1,0.80,0.86,5,262,6,0,1,0,0,2
2,0.11,0.88,7,272,4,0,1,0,0,2
3,0.72,0.87,5,223,5,0,1,0,0,1
4,0.37,0.52,2,159,3,0,1,0,0,1


Categorias: 
- Nível de Satisfação do Funcionário (satisfacao)
- Avaliação do Funcionário (evaluacao)
- Número de projetos do Funcionário (projetos)
- Média de horas gastas no trabalho (hrsmedia)
- Tempo gasto na empresa (tempogasto)
- Teve (1) ou não (0) um acidente de trabalho (acidentes)
- Teve (1) ou não (0) uma promoção nos últimos 5 anos (promocoes)
- Departamento de trabalho do Funcionário (setor)
(0) - Vendas  |  (1) - Contabilidade  |  (2) - RH  |  (3) - Técnico  |  (4) - Suporte  |  (5) - Diretoria  |  (6) - TI  |  (7) - Gerenciamento de Produto  |  (8) - Marketing
- Salário do Funcionário (salario)
(1) - baixo | (2) - médio | (3) - alto

- Saiu (1) ou não (0) da empresa (saiu)

In [333]:
#variaveis setor e salario sao categoricas, por isso é necessario realizar essa adaptacao mostrada no tutorial

y, X = dmatrices('saiu ~ satisfacao + evaluacao + projetos + hrsmedia + \
                  tempogasto + C(acidentes) + C(promocoes) + C(salario) + C(setor)',
                  data, return_type="dataframe")



print (X.columns)

Index(['Intercept', 'C(acidentes)[T.1]', 'C(promocoes)[T.1]',
       'C(salario)[T.2]', 'C(salario)[T.3]', 'C(setor)[T.1]', 'C(setor)[T.2]',
       'C(setor)[T.3]', 'C(setor)[T.4]', 'C(setor)[T.5]', 'C(setor)[T.6]',
       'C(setor)[T.7]', 'C(setor)[T.8]', 'C(setor)[T.RandD]', 'satisfacao',
       'evaluacao', 'projetos', 'hrsmedia', 'tempogasto'],
      dtype='object')


In [334]:
# mudar o nome das colunas de X como mostrado no tutorial, apenas para ficar mais facil identificar
X = X.rename(columns = {'C(setor)[T.1]':'vendas',
                        'C(setor)[T.2]':'contabilidade',
                        'C(setor)[T.3]':'RH',
                        'C(setor)[T.4]':'tecnico',
                        'C(setor)[T.5]':'suporte',
                        'C(setor)[T.6]':'diretoria',
                        'C(setor)[T.7]':'TI',
                        'C(setor)[T.8]':'produto',
                        'C(salario)[T.2]':'salario baixo',
                        'C(salario)[T.3]':'salario medio',
                        'C(acidentes)[T.1]':'acidentes',
                        'C(promocoes)[T.1]':'promocoes'
                       })

In [335]:
# metodo descrito pelo tutorial especifico dessa biblioteca e descrito como 'flatten y into a 1-D array'
y = np.ravel(y)

In [336]:
# atribuir a uma variável 'model' os parametros e coeficientes calculados a partir do comando 'Logistic Regression'
model = LogisticRegression(penalty='l2', )
model = model.fit(X, y)

# checar a precisao do modelo com o comando 'score'
model.score(X, y)

0.79158610574038268

In [337]:
# qual porcentagem real de funcionarios saiu da empresa
y.mean()

0.2380825388359224

##### Precisão

Foi obtida uma precisão de 79,15% com esse algoritmo que a princípio seria muito boa, uma vez que o objetivo principal é tentar prever quais funcionários sairão da empresa para evitar sua saída ou pelo menos se prevenir para esse possível evento.

No entanto, ao se analisar a porcentagem real de funcionários que saíram da empresa, 23%, percebe-se que uma precisão de 77% teria sido atingida caso o algoritmo sempre previsse que o funcionário não iria sair. Assim, a precisão do progama não é muito maior do que a precisão obtida caso ele sempre rejeitasse a hipótese de o funcionário sairá da empresa. Tentaremos a seguir melhorar essa precisão a partir da retirada de certas variáveis.

##### Coeficientes

In [338]:
# Checar os coeficientes entre sair da empresa (Y) e as outras variaveis (X)
#negativo indica que quanto maior a variavel menor a chance de sair da empresa
#se for positivo quanto menor um menor a chance do outro

pd.DataFrame(list(zip(X.columns, np.transpose(model.coef_))))

,0,1
0,Intercept,[0.19744678158]
1,acidentes,[-1.51995108815]
2,promocoes,[-1.17978480391]
3,salario baixo,[-0.526250226356]
4,salario medio,[-1.88508137238]
5,vendas,[0.0372154898282]
6,contabilidade,[0.27258953843]
7,RH,[0.105127234278]
8,tecnico,[0.0887136293211]
9,suporte,[-0.409211320196]


- Como é possível análisar pelos coeficientes, o setor de trabalho com menor chance de saida do funcionário é o de Suporte, já que ele apresenta um coeficiente negativo e com o maior módulo entre os setores.
- Além disso, aumentos na satisfação e no número de projetos feitos correspondem a uma redução na probabilidade do empregado se demitir.
- Pode-se também perceber que a ocorrência de um acidente ou promoção aumenta as chances do funcionário não sair da empresa.
- Por fim, parece que as horas médias não tem uma influência considerável na variável Y e quanto maior o tempo gasto e a avaliação do funcionnário maior as chances dele sair da empresa.

É importante notar que a classificação de um funcionário como provável de sair ou não depende do conjunto desses coeficientes, não apenas de cada um separado, então não há como confirmar ou rejeitar a hipótese apenas considerando uma variável.

##### Teste e treinamento

Para verificar se a precisão do modelo está correta é preciso separar a base de dados em duas, uma de testes e uma de treinamento. Assim, será possível definir se a precisão anterior foi atingida porque o algoritmo realmente tem essa taxa de acerto ou porque ele foi testado com a mesma base de dados em que foi treinado.

In [339]:
# avaliar o modelo dividindo em bases de treinamento (train) e testes (test)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

model2 = LogisticRegression()
model2.fit(X_train, y_train)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [340]:
# generate evaluation metrics
print (metrics.accuracy_score(y_test, predicted))

0.790888888889


A precisão obtida divindindo o dataframe em teste e treinamento foi de 79%, que é quase igual a conseguida anteriormente. Portanto, a precisão geral anterior estava correta.

In [341]:
# A função confusion_matrix() calcula a matriz de confusão e retorna o resultado como uma array
predicted = model2.predict(X_test)
print (metrics.confusion_matrix(y_test, predicted))


[[3203  259]
 [ 682  356]]


 O resultado mostra que há 3203 + 356 previsões corretas e 682 + 259 previsões incorretas

In [342]:
probs = model2.predict_proba(X_test)
print (metrics.classification_report(y_test, predicted))

             precision    recall  f1-score   support

        0.0       0.82      0.93      0.87      3462
        1.0       0.58      0.34      0.43      1038

avg / total       0.77      0.79      0.77      4500



Precision:
- Daqueles classificados como que vão sair, qual proporção realmente saiu?
- Verdadeiro positivo / (Verdadeiro positivo + Falso positivo)

Recall:
- Daqueles que sairam de fato, qual proporção foi classificada como tal?
- Verdadeiro positivo / (Verdadeiro positivo + Falso negativo)

A precision seria um tipo de precisão do modelo, porém considerando apenas os dados previstos como positivos, ou seja, com o cliente tendo saido da empresa. O recall também seria um tipo de precisão, mas considerando apenas dados que são relevantes de alguma maneira. Resumindo, são duas medidas que indicam a qualidade de um modelo de classificação binária.

O f1-score pode ser interpretado como uma média ponderada da precision e do recall, em que ele atinge seu melhor valor em 1 e o pior em 0.

Interpretando os resultados, pode-se afirmar que do modelo inteiro, 77% dos previstos como que vão sair, sairam mesmo. E 79% dos que sairam de fato foram previstos pelo programa como que sairam. 

##### Uso Prático em um App

O software RH Analysis utiliza um algoritmo baseado em regressão logística para fazer balanceamento de demissão (do próprio ou pedido pela empresa) dos funcionários da empresa.
 
O diretor de RH pode utilizar o software de duas maneiras mais intuitivas. 

- Análise Cirurgica

Esse tipo de análise mostra para o diretor um modo de analisar individualmente cada funcionário e tem mais efetividade para resultados à curto prazo. Nesse modelo, o diretor completa os campos requisitados com as características do funcionário que estão descritas abaixo, e o software lhe retorna os dados da probabilidade deste deixar a empresa ou não, baseado nas características previamente preenchidas. 

Características Requisitadas:
Nível de Satisfação do Funcionário
Avaliação do Funcionário 
Número de projetos do Funcionário 
Média de horas gastas no trabalho 
Tempo gasto na empresa 
Teve ou não um acidente de trabalho 
Teve ou não uma promoção nos últimos 5 anos 
Departamento de trabalho do Funcionário 
Salário do Funcionário 



- Análise Geral

Esse modelo requer que o diretor invista seu tempo primeiramente em uma base de dados. Essa base de dados deve conter, para cada um dos funcionários da empresa, cada uma das características colocadas acima, preenchidas. 
Uma vez que a base de dados está pronta, basta roda-lá no software RH Analisys e este irá varrer cada um dos funcionários utilizando os dados inseridos, e apurar a probabilidade de cada um deixar ou não a empresa. Consequentemente, o software analisa no ambito macroscópico a saída dos funcionários da empresa.

#### Exemplo da Análise cirurgica

Para simular o algoritmo da análise cirurgica podemos prever a probabilidade de um funcionário especificado sair da empresa. Para isso, basta forncer ao programa as características importantes do funcionário.

Dados: 
- Satisfação: 0.11
- Evaluação: 0.71
- Projetos: 2
- Horas médias: 267
- Tempo gasto: 5
- Acidentes: 0
- Promoções: 0
- Setor: 0
- Salário: 0


In [343]:
model.predict_proba(np.array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0.11, 0.71, 2,
                              267, 5]))

C:\Users\elijo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.15906298,  0.84093702]])

Como é possível observar, a probabilidade do funcionário sair da empresa é de 84%, assim considerando o modelo binário do algoritmo, ele sairia da empresa. 

#### Observação sobre a Precisão:

Um método que poderia ser utilizado para tentar melhorar a precisão do modelo seria mudar o valor do cutoff baseado na proporção entre o precision e recall, atingindo uma precisão melhor. No entanto, mesmo se esse valor ideal fosse encontrado, não seria possível aplica-lo eficientemente no algoritimo, já que pelo método usado (LogisticRegression()) ele não pode ser inserido ou modificado um parâmetro. Assim, só seria possível testar isso posteriormente pelo método predict, o que apenas daria a precisão do modelo sem fornecer quais funcionarios efetivamente sairiam ou não.

### Conclusão

Então, apesar do modelo desenvolvido não estar muito preciso, ele poderia ser utilizado em um programa para prever superficialmente quais funcionários poderiam acabar saindo da empresa. Assim, mesmo errando muito, ele demonstra que é possível realizar um algoritmo de regressão logística para a saida de funcionários. Além disso, ele ajuda parcialmente a responder a pergunta, uma vez que ajuda o setor de RH a ter uma melhor ideia da causas da demissão própria de empregador e a se prevenir em caso de funcionário que acabe, efetivamente deixando a empresa. Portanto, é possível concluir que com o emprego de técnicas de aperfeiçoamento e o uso de outras bibliotecas de regressão do python, seria possível melhorar a precisão do algoritmo.

#### Divisão de trabalho:

- Eli -  programação do algoritmo (com a ajuda de Pedro Greco para solucionar erros e dificuldades)

- Pedro Greco -  Análises sobre o algoritmo e sobre os resultados encontrados e explicação de regressão logística (junto com Eduardo Vaz)

- Eduardo Vaz -  Uso prático em um App e explicação de regressão logística (junto com Pedro Greco)

obs: O GitHub não foi utilizados por todos, as mudanças e edições do trabalho foram enviadas entre os integrantes por WhatsApp. Por isso, o GitHub só foi usado para fazer o upload da edição final do notebook a cada check e não como ferramenta de compartilhamento de edição.